In [1]:
import os
import sys
import glob

import pandas as pd
import numpy as np

import yaml

import re

import shutil

In [2]:
strCurrentOutputsDir = '/archive/bioinformatics/DLLab/AlexTreacher/data/20210825_ABIDE_CPAC/ABIDEI/derivatives/CPAC/output/cpac_cpac-custom-nuisance'
strDataConfigFile = '/archive/bioinformatics/DLLab/AlexTreacher/src/autism_DCG/cpac_pipeline/20210825_ABIDE_CPAC/ABDIEI/cpac_data_config_2021-08-27T17-10-10Z.yml'

In [3]:
with open(strDataConfigFile) as f:
    dctYMLConfig = yaml.safe_load(f)

In [4]:
dctYMLConfig.__len__()

1102

In [5]:
for dctSubject in dctYMLConfig:
    print(dctSubject['anat'])

/project/bioinformatics/DLLab/STUDIES/ABIDE1/source/sub-0050418/ses-1/anat/sub-0050418_ses-1_run-1_T1w.nii.gz
/project/bioinformatics/DLLab/STUDIES/ABIDE1/source/sub-0051018/ses-1/anat/sub-0051018_ses-1_run-1_T1w.nii.gz
/project/bioinformatics/DLLab/STUDIES/ABIDE1/source/sub-0051150/ses-1/anat/sub-0051150_ses-1_run-1_T1w.nii.gz
/project/bioinformatics/DLLab/STUDIES/ABIDE1/source/sub-0050653/ses-1/anat/sub-0050653_ses-1_run-1_T1w.nii.gz
/project/bioinformatics/DLLab/STUDIES/ABIDE1/source/sub-0051207/ses-1/anat/sub-0051207_ses-1_run-1_T1w.nii.gz
/project/bioinformatics/DLLab/STUDIES/ABIDE1/source/sub-0050315/ses-1/anat/sub-0050315_ses-1_run-1_T1w.nii.gz
/project/bioinformatics/DLLab/STUDIES/ABIDE1/source/sub-0051488/ses-1/anat/sub-0051488_ses-1_run-1_T1w.nii.gz
/project/bioinformatics/DLLab/STUDIES/ABIDE1/source/sub-0050130/ses-1/anat/sub-0050130_ses-1_run-1_T1w.nii.gz
/project/bioinformatics/DLLab/STUDIES/ABIDE1/source/sub-0051576/ses-1/anat/sub-0051576_ses-1_run-1_T1w.nii.gz
/project/b

In [6]:
# next cell below is used to remove incomplte runs from the output (e.g. due to a job being cancled) 

In [7]:
strOutputDir = strCurrentOutputsDir
lOutputs = os.listdir(strOutputDir)

lIncomplete = []
for strDir in lOutputs:
    lReHoOutputs = glob.glob(os.path.join(strOutputDir,strDir,'func','*reho.nii.gz'))
    if lReHoOutputs.__len__() < 1:
        lIncomplete.append(os.path.join(strOutputDir,strDir))

#uncomment this next part to move the failed subjects to del
for strPath in lIncomplete:
    shutil.move(strPath, '/archive/bioinformatics/DLLab/AlexTreacher/temp/del')

In [8]:
lIncomplete.__len__()

137

### Two methods of limiting CPAC
1. create a data YML with only the subjects we want ran
2. create a list of ndx to run, and only run the subjects that correspond to the subjects that correspond to those idx

lets go with option 2, that way it's kind of the same "job" and we just run until completion

In [9]:
lNDX = list(range(dctYMLConfig.__len__()))

In [10]:
#get the NDX that have already ran

In [11]:
lCompletedScans = os.listdir(strCurrentOutputsDir)

In [12]:
lNDXToIgnore = []
for strScan in lCompletedScans:
    for intNDX in lNDX:
        if dctYMLConfig[intNDX]['anat'].split(os.sep)[-1].startswith(strScan):
            lNDXToIgnore.append(intNDX)
lNDXToIgnore.__len__()

965

In [13]:
#add the subjects that are not in the matching lists

In [14]:
lNDXToRun = [x for x in lNDX if not x in lNDXToIgnore]

In [15]:
#save the NDX to run as a tsv (for GNU parallel to read)
pd.Series(lNDXToRun).to_csv('DNX_to_run.tsv', sep='\t', index=False, header=False)

In [16]:
lNDXToRun.__len__()

137